In [1]:
from jax import random
import jax
import neural_tangents as nt
from neural_tangents import stax

2023-02-17 08:55:00.105153: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(10,parameterization = 'ntk')
)

In [3]:
n_train = 3
n_test = 1
d = 100 
key1, key2, key3 = random.split(random.PRNGKey(1), num=3)
output_shape, params = init_fn(key1, input_shape=(d,))
delta = jax.numpy.sqrt(d/n_train) + 10
cov = jax.numpy.identity(d, dtype=None)
cov = cov.at[0,0].set(1+delta)
mean = jax.numpy.zeros(d)

2023-02-17 08:55:17.861268: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:397] There was an error before creating cudnn handle: cudaGetErrorName symbol not found. : cudaGetErrorString symbol not found.
2023-02-17 08:55:17.861407: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:429] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-02-17 08:55:17.862945: E external/org_tensorflow/tensorflow/compiler/xla/status_macros.cc:57] INTERNAL: RET_CHECK failure (external/org_tensorflow/tensorflow/compiler/xla/service/gpu/gpu_compiler.cc:626) dnn != nullptr 
*** Begin stack trace ***
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	PyCFunction_Call
	_PyObject_MakeTpCall
	
	_PyEval_EvalFrameDefault
	_PyFunction_Vectorcall
	PyObject_Call
	_PyEval_EvalFrameDefault
	_PyEval_EvalCodeWithName
	_PyFunction_Vectorcall
	_PyEval_EvalFrameDefault
	_PyFunction_Vectorcall
	_PyEval_EvalFrameDefault
	_PyEval_EvalCodeWithName
	_PyFunction_Vecto

XlaRuntimeError: INTERNAL: RET_CHECK failure (external/org_tensorflow/tensorflow/compiler/xla/service/gpu/gpu_compiler.cc:626) dnn != nullptr 

In [ ]:
x_train = jax.random.multivariate_normal(key2, mean, cov, (n_train,))
y_train = random.uniform(key2, shape=(n_train, 1))  # training targets
x_test = jax.random.multivariate_normal(key3, mean, cov, (n_test,))

In [ ]:
def returnsNTkKernel(x_train, kernel_fn, x=None, train=True):
    if train:
        ntk_train_train = kernel_fn(x_train, x_train, 'ntk')
        return ntk_train_train
    ntk_test_train = kernel_fn(x, x_train, 'ntk')
    return ntk_test_train

def EigenDecompositonOfKernel(ntk_train_train):
    eigenvalues, eigenvectors = jax.numpy.linalg.eigh(ntk_train_train)
    return eigenvalues, eigenvectors

def feature(y_train, ntk_test_train, eigenvalues, eigenvectors, i):
    lambda_i, v_i = eigenvalues[i], eigenvectors[i].reshape(eigenvectors[i].shape[0],1)
    output = ntk_test_train @ v_i @ v_i.T @ y_train
    print(output, lambda_i, v_i.T@v_i)
    output = output/lambda_i
    return output

def returnsXPerturbed(x_test, epsilon, key):
    noise = random.normal(key, x_test.shape)
    noise_norm = jax.numpy.linalg.norm(noise, axis=1)
    noise_norm = noise_norm.reshape(noise_norm.shape[0],1)
    normed_noise = epsilon*noise/noise_norm
    x_perturbed = x_test+normed_noise
    return x_perturbed

In [ ]:
ntk_train_train = returnsNTkKernel(x_train, kernel_fn, x=None, train=True)
eigenvalues, eigenvectors = EigenDecompositonOfKernel(ntk_train_train)

In [ ]:
cov_eigenvalues, cov_eigenvectors = EigenDecompositonOfKernel((1/d)*x_train@x_train.T)

In [ ]:
norms={}
ntk_test_train_clean = returnsNTkKernel(x_train, kernel_fn, x_test, train=False)
for epsilon in [0.5, 0.7, 1, 2]:
    norms[str(epsilon)] = []
    x_perturbed = returnsXPerturbed(x_test, epsilon, key3)
    ntk_test_train_perturbed = returnsNTkKernel(x_train, kernel_fn, x_perturbed, train=False)   
    fx_delta = lambda x: abs(feature(y_train, ntk_test_train_perturbed, eigenvalues, eigenvectors, x))
    fx_clean = lambda x: abs(feature(y_train, ntk_test_train_clean, eigenvalues, eigenvectors, x))
    output_delta = map(fx_delta, range(n_train))
    output_clean = map(fx_clean, range(n_train))
    difference =  jax.numpy.array(list(output_delta))-jax.numpy.array(list(output_clean))
    norms[str(epsilon)] = difference.reshape(difference.shape[0])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
fig = plt.figure(figsize=(15,5))
gs = matplotlib.gridspec.GridSpec(1, 2, width_ratios=[2, 2]) 
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
for epsilon in norms.keys():
    ax1.plot(eigenvalues, abs(norms[epsilon]), label=epsilon)
ax2.plot(eigenvalues, marker='+', label="kernel")
ax2.plot(cov_eigenvalues, marker='.', label="cov")
ax1.set_ylabel("||f_x_perturbed-f_x||")
ax1.set_xlabel("Eigenvalues")
ax2.set_ylabel("Eigenvalues")
#ax1.set_yscale("log")
ax1.legend()
ax2.legend()
plt.show()

In [ ]:
eigenvalues

In [8]:
!pip uninstall neural_tangents --yes

Found existing installation: neural-tangents 0.6.2
Uninstalling neural-tangents-0.6.2:
  Successfully uninstalled neural-tangents-0.6.2
